# The Movie DataBase (TMDB)

In [1]:
import datetime
import pandas as pd
import numpy as np
import json
import re

In [70]:
movies = pd.read_csv('tmdb_5000_movies.csv')
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [3]:
credits = pd.read_csv('tmdb_5000_credits.csv')
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


1- For each movie, compute the number of cast members

In [4]:
def cast_members(data):
    item_dict = json.loads(data)
    return(len(item_dict))
credits['cast_members'] = credits['cast'].apply(cast_members)
credits[['movie_id','title','cast_members']].head()

,movie_id,title,cast_members
0,19995,Avatar,83
1,285,Pirates of the Caribbean: At World's End,34
2,206647,Spectre,83
3,49026,The Dark Knight Rises,158
4,49529,John Carter,27


2- How many movies do not have a homepage?

In [11]:
nohomepage = movies[movies['homepage'].isnull()]
conteggio = len(nohomepage.index)
print('I film che non hanno una homepage sono',conteggio)

I film che non hanno una homepage sono 3091


3- For each year, how many movies do not have a homepage?

In [13]:
movies['years'] = pd.to_datetime(movies['release_date'],format="%Y-%m-%d").dt.year   #creazione var anno
movies['years'] = movies['years'].replace(np.nan, 'Missing')
nohomepage = movies[movies['homepage'].isnull()]#selezione film senza homepage
nohomepage[['id', 'years']].groupby('years').count().tail().rename(index=str, columns={'id':'count_noHomepage'})

,count_noHomepage
years,
2013.0,127
2014.0,157
2015.0,110
2016.0,31
Missing,1


4- Extract the domain of each homepage.

In [131]:
def extract_dom(url):
    dom = []
    if (pd.isnull(url)):
        return(np.nan)
    else:
        homepage = url.split(' ')                     #per poter gestire i film con due siti
        for i in range(len(homepage)):
            result=re.findall(r'[\w\-+.]+',homepage[i])
            dom.append(result[1])
        return str(dom)[1:-1].replace("'", "") 
movies['domain']=movies['homepage'].apply(extract_dom)
movies[['title', 'homepage', 'domain']][3730:3740]

,title,homepage,domain
3730,Cargo,http://www.cargoderfilm.ch http://cargothemovi...,"www.cargoderfilm.ch, cargothemovie.com"
3731,High School Musical,http://tv.disney.go.com/disneychannel/original...,tv.disney.go.com
3732,Love and Death on Long Island,NaN,NaN
3733,Night Watch,NaN,NaN
3734,The Crying Game,http://www.miramax.com/movie/the-crying-game/,www.miramax.com
3735,Porky's,NaN,NaN
3736,Survival of the Dead,http://magnetreleasing.com/survivalofthedead/,magnetreleasing.com
3737,Night of the Living Dead,NaN,NaN
3738,Lost in Translation,NaN,NaN
3739,Annie Hall,NaN,NaN


5- Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).

#### Creo tabella movie_id

In [132]:
movie_id = movies[['id', 'budget', 'homepage', 'domain', 'original_language', 'original_title', 'overview', 'popularity', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count' ]]
movie_id = movie_id.set_index(['id'])
movie_id.head(2)

,budget,homepage,domain,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count
id,,,,,,,,,,,,,,,
19995,237000000,http://www.avatarmovie.com/,www.avatarmovie.com,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2787965087,162.0,Released,Enter the World of Pandora.,Avatar,7.2,11800
285,300000000,http://disney.go.com/disneypictures/pirates/,disney.go.com,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,961000000,169.0,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


#### Creo tabella genere

In [133]:
genere=pd.DataFrame()
for film in list(range(0,len(movies['genres']))):
    tabjson=pd.read_json(movies['genres'][film])
    genere=pd.concat([genere, tabjson])
genere = genere.set_index(['id'])
genere= genere.drop_duplicates() #per eliminare le righe duplicate
genere.head()

,name
id,
28,Action
12,Adventure
14,Fantasy
878,Science Fiction
80,Crime


#### Creo tabella genere_id

In [134]:
genere_id=pd.DataFrame()
for film in list(range(0,len(movies['genres']))):
    tabjson=pd.read_json(movies['genres'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    genere_id=pd.concat([genere_id, tabjson])
del genere_id['name']
genere_id = genere_id.set_index(['movie_id', 'id'])
genere_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 28.0), (19995, 12.0), (19995, 14.0), (19995, 878.0), (285, 12.0)]

#### Creo tabella keywords

In [135]:
keywords=pd.DataFrame()
for film in list(range(0,len(movies['keywords']))):
    tabjson=pd.read_json(movies['keywords'][film])
    keywords=pd.concat([keywords, tabjson])
keywords = keywords.set_index(['id'])
keywords = keywords.drop_duplicates()
keywords.head()

,name
id,
1463,culture clash
2964,future
3386,space war
3388,space colony
3679,society


#### Creo tabella keywords_id

In [136]:
keywords_id=pd.DataFrame()
for film in list(range(0,len(movies['keywords']))):
    tabjson=pd.read_json(movies['keywords'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    keywords_id=pd.concat([keywords_id, tabjson])
del keywords_id['name']
keywords_id = keywords_id.set_index(['movie_id', 'id'])
keywords_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 1463.0), (19995, 2964.0), (19995, 3386.0), (19995, 3388.0), (19995, 3679.0)]

#### Creo tabella prod_companies

In [137]:
prod_companies=pd.DataFrame()
for film in list(range(0,len(movies['production_companies']))):
    tabjson=pd.read_json(movies['production_companies'][film])
    prod_companies=pd.concat([prod_companies, tabjson])
prod_companies = prod_companies.set_index(['id'])
prod_companies = prod_companies.drop_duplicates()
prod_companies.head()

,name
id,
289,Ingenious Film Partners
306,Twentieth Century Fox Film Corporation
444,Dune Entertainment
574,Lightstorm Entertainment
2,Walt Disney Pictures


#### Creo tabella prod_companies_id

In [138]:
prod_companies_id=pd.DataFrame()
for film in list(range(0,len(movies['production_companies']))):
    tabjson=pd.read_json(movies['production_companies'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    prod_companies_id=pd.concat([prod_companies_id, tabjson])
del prod_companies_id['name']
prod_companies_id = prod_companies_id.set_index(['movie_id', 'id'])
prod_companies_id.head()

Empty DataFrame
Columns: []
Index: [(19995, 289.0), (19995, 306.0), (19995, 444.0), (19995, 574.0), (285, 2.0)]

#### Creo tabella prod_countries

In [139]:
prod_countries=pd.DataFrame()
for film in list(range(0,len(movies['production_countries']))):
    tabjson=pd.read_json(movies['production_countries'][film])
    prod_countries=pd.concat([prod_countries, tabjson])
prod_countries = prod_countries.set_index(['iso_3166_1'])
prod_countries = prod_countries.drop_duplicates()
prod_countries.head()

,name
iso_3166_1,
US,United States of America
GB,United Kingdom
JM,Jamaica
BS,Bahamas
DM,Dominica


#### Creo tabella prod_countries_id

In [140]:
prod_countries_id=pd.DataFrame()
for film in list(range(0,len(movies['production_countries']))):
    tabjson=pd.read_json(movies['production_countries'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    prod_countries_id=pd.concat([prod_countries_id, tabjson])
del prod_countries_id['name']
prod_countries_id = prod_countries_id.set_index(['movie_id', 'iso_3166_1'])
prod_countries_id.head()

Empty DataFrame
Columns: []
Index: [(19995, US), (19995, GB), (285, US), (206647, GB), (206647, US)]

#### Creo tabella spoken_languages

In [141]:
spoken_languages=pd.DataFrame()
for film in list(range(0,len(movies['spoken_languages']))):
    tabjson=pd.read_json(movies['spoken_languages'][film])
    spoken_languages=pd.concat([spoken_languages, tabjson])
spoken_languages = spoken_languages.set_index(['iso_639_1'])
spoken_languages = spoken_languages.drop_duplicates()
spoken_languages.head()

,name
iso_639_1,
en,English
es,Español
fr,Français
it,Italiano
de,Deutsch


#### Creo tabella spoken_languages_id

In [142]:
spoken_languages_id=pd.DataFrame()
for film in list(range(0,len(movies['spoken_languages']))):
    tabjson=pd.read_json(movies['spoken_languages'][film])
    idfilm=movies['id'][film]
    tabjson['movie_id']=idfilm
    spoken_languages_id=pd.concat([spoken_languages_id, tabjson])
del spoken_languages_id['name']
spoken_languages_id = spoken_languages_id.set_index(['movie_id', 'iso_639_1'])
spoken_languages_id.head()

Empty DataFrame
Columns: []
Index: [(19995, en), (19995, es), (285, en), (206647, fr), (206647, en)]

#### Creo tabella credits_id

In [143]:
credits_id=credits[['movie_id', 'cast_members']]
credits_id = credits_id.set_index(['movie_id'])
credits_id.head()

,cast_members
movie_id,
19995,83
285,34
206647,83
49026,158
49529,27


#### Creo tabella cast_id

In [144]:
cast=pd.DataFrame()
for film in list(range(0,len(credits['cast']))):
    tabjson=pd.read_json(credits['cast'][film])
    idfilm=credits['movie_id'][film]
    tabjson['movie_id']=idfilm
    cast=pd.concat([cast, tabjson])
cast_id = cast[['movie_id', 'cast_id', 'id', 'order', 'credit_id']]
cast_id = cast_id.set_index(['movie_id','cast_id'])
cast_id.head()

id  order                 credit_id
movie_id cast_id                                          
19995    242.0    65731.0    0.0  5602a8a7c3a3685532001c9a
         3.0       8691.0    1.0  52fe48009251416c750ac9cb
         25.0     10205.0    2.0  52fe48009251416c750aca39
         4.0      32747.0    3.0  52fe48009251416c750ac9cf
         5.0      17647.0    4.0  52fe48009251416c750ac9d3

#### Creo tabella crew_id

In [145]:
crew=pd.DataFrame()
for film in list(range(0,len(credits['crew']))):
    tabjson=pd.read_json(credits['crew'][film])
    idfilm=credits['movie_id'][film]
    tabjson['movie_id']=idfilm
    crew=pd.concat([crew, tabjson])
crew_id = crew[['movie_id', 'credit_id', 'id', 'department', 'job']]
crew_id = crew_id.set_index(['movie_id', 'credit_id'])
crew_id.head()

id  department  \
movie_id credit_id                                      
19995    52fe48009251416c750aca23  1721.0     Editing   
         539c47ecc3a36810e3001f87   496.0         Art   
         54491c89c3a3680fb4001cf7   900.0       Sound   
         54491cb70e0a267480001bd0   900.0       Sound   
         539c4a4cc3a36810c9002101  1262.0  Production   

                                                        job  
movie_id credit_id                                           
19995    52fe48009251416c750aca23                    Editor  
         539c47ecc3a36810e3001f87         Production Design  
         54491c89c3a3680fb4001cf7            Sound Designer  
         54491cb70e0a267480001bd0  Supervising Sound Editor  
         539c4a4cc3a36810c9002101                   Casting

#### Creo tabella attori

In [146]:
attori_crew=crew[['id', 'gender', 'name']]
attori_cast=cast[['id', 'gender', 'name']]
attori=pd.concat([attori_crew, attori_cast])
attori=attori.sort_values('id').drop_duplicates().set_index('id')
attori.head()

,gender,name
id,,
1.0,2.0,George Lucas
2.0,2.0,Mark Hamill
3.0,2.0,Harrison Ford
4.0,1.0,Carrie Fisher
5.0,2.0,Peter Cushing


6- For each movie, compute the gross margin (difference between revenue and budget)

In [147]:
movies['gross_margin'] = movies['revenue'] - movies['budget']
movies[['id','title','gross_margin']].set_index('id').head()

,title,gross_margin
id,,
19995,Avatar,2550965087
285,Pirates of the Caribbean: At World's End,661000000
206647,Spectre,635674609
49026,The Dark Knight Rises,834939099
49529,John Carter,24139100


7- For each movie, compute the number of crew members

In [163]:
members = crew[['movie_id','id']].groupby('movie_id').count() #si calcolano i membri per ciascun film
#per una miglior visualizzazione dei risultati si aggiunge la colonna 'title' e si rinomina la var id_x
members = pd.merge(members.reset_index(), movies[['id', 'title']], left_on='movie_id', right_on='id') 
members = members.rename(index=str, columns={"id_x" : "n_CrewMembers"})
members[['movie_id', 'title', 'n_CrewMembers']].set_index('movie_id').head()

,title,n_CrewMembers
movie_id,,
5,Four Rooms,88
11,Star Wars,20
12,Finding Nemo,104
13,Forrest Gump,93
14,American Beauty,109


8- For each movie, compute the number of directors

In [26]:
directors = crew.loc[crew.job == 'Director']
num_directors= directors[['movie_id', 'id']].groupby('movie_id').count()
num_directors.rename(index=str, columns={'id': 'n_Directors'}).head()

,n_Directors
movie_id,
5,4
11,1
12,1
13,1
14,1


9- For each language, compute the number of movies where such language is spoken.

In [27]:
spoken_languages_id= spoken_languages_id.reset_index()
spoken_languages= spoken_languages.reset_index()
df2= pd.merge(spoken_languages_id, spoken_languages, on = 'iso_639_1')
df2.head()

,movie_id,iso_639_1,name
0,19995,en,English
1,285,en,English
2,206647,en,English
3,49026,en,English
4,49529,en,English


In [28]:
lang= df2.groupby(['name', 'iso_639_1']).count()
lang.rename(index=str, columns={'movie_id': 'count_movie'})
lang= lang.iloc[2:] #vengono tolte le prime due righe del dataset, non classificabili in nessuna lingua
lang= lang.rename(index=str, columns={'movie_id':'n_Movie'})
lang.head()

,,n_Movie
name,iso_639_1,
Afrikaans,af,7
Bahasa indonesia,id,2
Bamanankan,bm,1
Bosanski,bs,2
Català,ca,1


10- For each company and each decade, compute the overall revenue

In [29]:
pc= prod_companies_id.reset_index()
PC= pc.groupby('movie_id').count()
pc1= movies.reset_index()
pc1= pc1[['id', 'revenue', 'years']]
pc1['decade']= pd.to_datetime(movies['release_date'],format="%Y-%m-%d").dt.year//10
pc1['decade'] = pc1.decade.replace(np.nan, 'Missing')
pc2=pd.merge(PC.reset_index(), pc1, left_on= 'movie_id', right_on='id')
pc3=pc2.rename(index=str, columns={'id_x':'count_companies'})[['movie_id', 'count_companies', 'revenue', 'decade']]
pc3['revenue/companies']= pc3['revenue']/pc3['count_companies']
ALL= pd.merge(pc, pc3, on='movie_id')
ALL.head()

,movie_id,id,count_companies,revenue,decade,revenue/companies
0,19995,289.0,4,2787965087,200,6.969913e+08
1,19995,306.0,4,2787965087,200,6.969913e+08
2,19995,444.0,4,2787965087,200,6.969913e+08
3,19995,574.0,4,2787965087,200,6.969913e+08
4,285,2.0,3,961000000,200,3.203333e+08


In [30]:
ALL_grouped= ALL[['id', 'decade','revenue/companies']].groupby(['id','decade']).sum().rename(index=str, columns={'revenue/companies':'overall_revenue'})
ALL_grouped.head()

overall_revenue
id  decade                 
1.0 197.0      4.343657e+08
    198.0      1.187157e+09
    199.0      9.243176e+08
    200.0      1.892716e+09
    201.0      1.678846e+07

11- For each decade, compute the company with maximum revenue

In [51]:
ALL_grouped3= ALL[['id', 'decade','revenue/companies']].groupby(['decade', 'id']).sum().rename(index=str, columns={'revenue/companies':'overall_revenue'})
ALL_grouped3= ALL_grouped3.groupby('decade').max()
pd.merge(ALL_grouped3.reset_index(), ALL_grouped.reset_index(), on=['decade', 'overall_revenue'])


,decade,overall_revenue,id
0,191.0,4.197376e+06,1307.0
1,191.0,4.197376e+06,1308.0
2,192.0,2.635800e+07,8411.0
3,193.0,2.000882e+08,1553.0
4,193.0,2.000882e+08,8411.0
5,194.0,3.095972e+08,3166.0
6,195.0,6.850000e+07,306.0
7,196.0,3.072341e+08,7576.0
8,197.0,7.673303e+08,33.0
9,198.0,3.082731e+09,4.0


12- In each year, how many movies have revenue smaller than the budget?

In [32]:
movies_unsuccess= movies.loc[movies.gross_margin < 0, ['years','gross_margin']]
movies_unsuccess= movies_unsuccess.groupby('years').count()
movies_unsuccess.rename(index=str, columns={'gross_margin': 'count_unsuccess'}).head()

,count_unsuccess
years,
1927.0,1
1947.0,1
1948.0,1
1954.0,1
1962.0,1


1- Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

In [33]:
data= pd.merge(cast.reset_index(), credits_id.reset_index(), on='movie_id')   #preparazione data set
data= pd.merge(data, movie_id.reset_index(), left_on='movie_id', right_on='id')
data=data[['movie_id', 'id_x', 'order', 'cast_members', 'revenue', 'title', 'name', 'character']]
data.head()

,movie_id,id_x,order,cast_members,revenue,title,name,character
0,19995,65731.0,0.0,83,2787965087,Avatar,Sam Worthington,Jake Sully
1,19995,8691.0,1.0,83,2787965087,Avatar,Zoe Saldana,Neytiri
2,19995,10205.0,2.0,83,2787965087,Avatar,Sigourney Weaver,Dr. Grace Augustine
3,19995,32747.0,3.0,83,2787965087,Avatar,Stephen Lang,Col. Quaritch
4,19995,17647.0,4.0,83,2787965087,Avatar,Michelle Rodriguez,Trudy Chacon


In [34]:
def parti(persone, incasso, ordine):
    somma=0
    massimo=1
    for i in list(range(0,persone)):    #calcola le parti per cui dividere il totale
        somma=somma + 2**(i)
    parte=(incasso/somma)*(2**(persone-ordine-1))   
    return(parte)
data['guadagno']=list(map(parti,data['cast_members'], data['revenue'], data['order']))
#per una migliore presentazione dei dati
guadagno=data[['title', 'revenue', 'id_x','name', 'character', 'order', 'guadagno']]
guadagno=guadagno.rename(index=str, columns={"id_x" : "id"})
guadagno.head()

,title,revenue,id,name,character,order,guadagno
0,Avatar,2787965087,65731.0,Sam Worthington,Jake Sully,0.0,1.393983e+09
1,Avatar,2787965087,8691.0,Zoe Saldana,Neytiri,1.0,6.969913e+08
2,Avatar,2787965087,10205.0,Sigourney Weaver,Dr. Grace Augustine,2.0,3.484956e+08
3,Avatar,2787965087,32747.0,Stephen Lang,Col. Quaritch,3.0,1.742478e+08
4,Avatar,2787965087,17647.0,Michelle Rodriguez,Trudy Chacon,4.0,8.712391e+07


2- For each actor find the total revenue attributed to him/her

In [35]:
df=guadagno[['id', 'guadagno']].groupby('id').sum()    #sommo incassi per attore
df=pd.merge(df.reset_index(), attori.reset_index(), on='id')   #aggiungo il nome dell'attore
df= df[['id', 'name', 'guadagno']].set_index('id')
df.head()

,name,guadagno
id,,
1.0,George Lucas,5.487439e+01
2.0,Mark Hamill,9.432493e+08
3.0,Harrison Ford,2.509048e+09
4.0,Carrie Fisher,2.483156e+08
5.0,Peter Cushing,4.846238e+07


3- Find the actor that is responsible for the most overall revenue

In [36]:
df.loc[df['guadagno'].idxmax()]

name         Tom Cruise
guadagno    3.97612e+09
Name: 500.0, dtype: object

1- For each movie, compute the ratio between males and females in the cast

In [52]:
merged = pd.merge(attori.reset_index(), cast_id.reset_index(), left_on='id', right_on='cast_id')
merged= merged[merged.gender != 0.0] #elimino i gender non assegnati (0.0)
grouped = merged.groupby(['movie_id', 'gender']).count()
counter = grouped.reset_index()
counter= counter[['movie_id', 'gender', 'id_x']]
counter.head()

,movie_id,gender,id_x
0,5,1.0,3
1,5,2.0,16
2,11,1.0,22
3,11,2.0,52
4,12,1.0,3


In [60]:
femmine=counter[counter.gender == 1.0]
maschi=counter[counter.gender == 2.0]
femmine=femmine.rename(index=str, columns={"id_x" : "femmine"})
maschi=maschi.rename(index=str, columns={"id_x" : "maschi"})
totale=pd.merge(femmine[['movie_id', 'femmine']], maschi[['movie_id', 'maschi']], on='movie_id')
totale['ratio']=totale['maschi']/totale['femmine']
totale.head()

,movie_id,femmine,maschi,ratio
0,5,3,16,5.333333
1,11,22,52,2.363636
2,12,3,18,6.000000
3,13,9,38,4.222222
4,14,7,28,4.000000


2- For each movie, compute the ratio between the attributed revenue of males and females in the cast

In [63]:
new_table = pd.merge(guadagno, attori.reset_index(), on='id')
new_table = new_table[['title', 'id', 'name_x', 'guadagno','gender']]
new_table = new_table[new_table['gender'] != 0.0]
new_table = new_table.groupby(['title', 'gender']).sum()
new_table = new_table.reset_index()[['title', 'gender', 'guadagno']]
new_table.head()

,title,gender,guadagno
0,#Horror,1.0,0.000000e+00
1,#Horror,2.0,0.000000e+00
2,(500) Days of Summer,1.0,2.362450e+07
3,(500) Days of Summer,2.0,3.709374e+07
4,10 Cloverfield Lane,1.0,5.504295e+07


In [64]:
guadagno_maschi = new_table[new_table['gender'] == 1.0]
guadagno_maschi = guadagno_maschi.reset_index()
guadagno_maschi = guadagno_maschi[['title', 'guadagno']]
guadagno_maschi = guadagno_maschi.rename(index=str, columns={"guadagno" : "guadagno_maschi"})
guadagno_maschi.head()

,title,guadagno_maschi
0,#Horror,0.000000e+00
1,(500) Days of Summer,2.362450e+07
2,10 Cloverfield Lane,5.504295e+07
3,10 Days in a Madhouse,0.000000e+00
4,10 Things I Hate About You,1.736475e+07


In [65]:
guadagno_femmine = new_table[new_table['gender'] == 2.0]
guadagno_femmine = guadagno_femmine.reset_index()
guadagno_femmine = guadagno_femmine[['title', 'guadagno']]
guadagno_femmine = guadagno_femmine.rename(index=str, columns={"guadagno" : "guadagno_femmine"})
guadagno_femmine.head()

,title,guadagno_femmine
0,#Horror,0.000000e+00
1,(500) Days of Summer,3.709374e+07
2,10 Cloverfield Lane,5.102058e+07
3,10 Days in a Madhouse,0.000000e+00
4,10 Things I Hate About You,3.610894e+07


In [67]:
tot_guadagni = pd.merge(guadagno_maschi, guadagno_femmine, on = 'title')
tot_guadagni['ratio_guadagno'] = tot_guadagni['guadagno_maschi'] / tot_guadagni['guadagno_femmine']
tot_guadagni.head()

,title,guadagno_maschi,guadagno_femmine,ratio_guadagno
0,#Horror,0.000000e+00,0.000000e+00,NaN
1,(500) Days of Summer,2.362450e+07,3.709374e+07,0.636887
2,10 Cloverfield Lane,5.504295e+07,5.102058e+07,1.078838
3,10 Days in a Madhouse,0.000000e+00,0.000000e+00,NaN
4,10 Things I Hate About You,1.736475e+07,3.610894e+07,0.480899


3- For each director, compute the average (among all movies he/she has directed) of the ratio found in the previous point.

In [70]:
mf_ratio_movies = pd.merge(movie_id.reset_index(),tot_guadagni, on = 'title')
mf_ratio_movies = mf_ratio_movies[['id', 'title', 'ratio_guadagno']]
mf_ratio_movies.head()

,id,title,ratio_guadagno
0,19995,Avatar,0.695379
1,285,Pirates of the Caribbean: At World's End,0.143497
2,206647,Spectre,0.196554
3,49026,The Dark Knight Rises,0.085903
4,49529,John Carter,0.601346


In [45]:
registi = crew_id[crew_id.job == 'Director']
nomi_registi = pd.merge(registi.reset_index(), attori.reset_index(), on = 'id')
nomi_registi = nomi_registi[['movie_id', 'id', 'name']]
nomi_registi.head()

,movie_id,id,name
0,19995,2710.0,James Cameron
1,597,2710.0,James Cameron
2,280,2710.0,James Cameron
3,36955,2710.0,James Cameron
4,2756,2710.0,James Cameron


In [71]:
merged_table = pd.merge(mf_ratio_movies, nomi_registi, left_on = 'id', right_on = 'movie_id')
merged_table = merged_table[['id_x', 'title', 'ratio_guadagno', 'id_y', 'name']]
merged_table = merged_table.rename(index=str, columns={'id_x' : 'movie_id', 'id_y' : 'director_id', 'name' : 'directors'})
merged_table.head()

,movie_id,title,ratio_guadagno,director_id,directors
0,19995,Avatar,0.695379,2710.0,James Cameron
1,285,Pirates of the Caribbean: At World's End,0.143497,1704.0,Gore Verbinski
2,206647,Spectre,0.196554,39.0,Sam Mendes
3,49026,The Dark Knight Rises,0.085903,525.0,Christopher Nolan
4,49529,John Carter,0.601346,7.0,Andrew Stanton


In [72]:
tot_ratio = merged_table.groupby('directors').sum()
tot_ratio = tot_ratio.reset_index()
tot_ratio = tot_ratio[['directors', 'ratio_guadagno']]
tot_ratio.head()

,directors,ratio_guadagno
0,Aaron Hann,NaN
1,Aaron Schneider,NaN
2,Aaron Seltzer,4.083812
3,Aaron T. Wells,NaN
4,Abel Ferrara,0.233276


In [48]:
tot_films = merged_table.groupby('directors').count()
tot_films = tot_films.reset_index()
tot_films = tot_films[['directors', 'movie_id']]
tot_films = tot_films.rename(index=str, columns={'movie_id' : 'n_films'})
tot_films.head()

,directors,n_films
0,Aaron Hann,1
1,Aaron Schneider,1
2,Aaron Seltzer,5
3,Aaron T. Wells,1
4,Abel Ferrara,1


In [74]:
final_table = pd.merge(tot_ratio, tot_films, on = 'directors')
final_table['average_ratio_guadagno'] = final_table['ratio_guadagno'] / final_table['n_films']
final_table.head()

,directors,ratio_guadagno,n_films,average_ratio_guadagno
0,Aaron Hann,NaN,1,NaN
1,Aaron Schneider,NaN,1,NaN
2,Aaron Seltzer,4.083812,5,0.816762
3,Aaron T. Wells,NaN,1,NaN
4,Abel Ferrara,0.233276,1,0.233276


4- Find the director that has the highest average computed in the previous point

In [75]:
max_average = final_table['average_ratio_guadagno'].max()
result = final_table[final_table['average_ratio_guadagno'] == max_average]
result

,directors,ratio_guadagno,n_films,average_ratio_guadagno
1141,Jonathan Glazer,524288.0,3,174762.666667


In [78]:
# informazione in più: film di Jonathan Glazer e ratio_guadagno associato
merged_table.loc[merged_table.directors== 'Jonathan Glazer']

,movie_id,title,ratio_guadagno,director_id,directors
2365,10740,Birth,NaN,66728.0,Jonathan Glazer
2850,97370,Under the Skin,524288.0,66728.0,Jonathan Glazer
3890,11826,Sexy Beast,NaN,66728.0,Jonathan Glazer
